In [3]:
library(SingleCellExperiment)
library(reticulate)
library(Seurat)
library(dplyr)
library(H5weaver)
library(SeuratDisk)
library(reticulate)
library(anndata)
library(reshape2)
library(tidyr)
library(limma)
library(edgeR)
library(pheatmap)
library(ggplot2)
library(gridExtra)
library(tibble)
library(plotly)
library(readxl)
library(VennDiagram)
library(fgsea)
library(UpSetR)
library(zip)
library(openxlsx)
library(statmod)

# load all H5AD files into R using Anndata

In [5]:
#load h5ad file with psb data
folder_path <- '/home/jupyter/IH-A-Aging-Analysis-Notebooks/Mansi_Notebooks/scRNA/Pathway_analysis/psb_subsets'
file_list <- list.files(path = folder_path, pattern = ".h5ad$", full.names = TRUE)

In [6]:
file_list

[1] "/home/jupyter/IH-A-Aging-Analysis-Notebooks/Mansi_Notebooks/scRNA/Pathway_analysis/psb_subsets/subset_C1Q+ CD16 monocyte.h5ad"   
[2] "/home/jupyter/IH-A-Aging-Analysis-Notebooks/Mansi_Notebooks/scRNA/Pathway_analysis/psb_subsets/subset_Core CD14 monocyte.h5ad"   
[3] "/home/jupyter/IH-A-Aging-Analysis-Notebooks/Mansi_Notebooks/scRNA/Pathway_analysis/psb_subsets/subset_Core CD16 monocyte.h5ad"   
[4] "/home/jupyter/IH-A-Aging-Analysis-Notebooks/Mansi_Notebooks/scRNA/Pathway_analysis/psb_subsets/subset_IL1B+ CD14 monocyte.h5ad"  
[5] "/home/jupyter/IH-A-Aging-Analysis-Notebooks/Mansi_Notebooks/scRNA/Pathway_analysis/psb_subsets/subset_Intermediate monocyte.h5ad"
[6] "/home/jupyter/IH-A-Aging-Analysis-Notebooks/Mansi_Notebooks/scRNA/Pathway_analysis/psb_subsets/subset_ISG+ CD14 monocyte.h5ad"   
[7] "/home/jupyter/IH-A-Aging-Analysis-Notebooks/Mansi_Notebooks/scRNA/Pathway_analysis/psb_subsets/subset_ISG+ CD16 monocyte.h5ad"

In [7]:
start_time <- Sys.time()
process_file <- function(file_path) {
  ad <- anndata::read_h5ad(file_path)
  data_matrix <- t(ad$X)
  data_matrix <- as.data.frame(data_matrix)
  rownames(data_matrix) <- ad$var_names
  colnames(data_matrix) <- ad$obs[['pbmc_sample_id']]
  return(data_matrix)
}
 
data_frames <- list()
for (file_path in file_list) {
  file_name <- tools::file_path_sans_ext(basename(file_path))
  data_frames[[file_name]] <- process_file(file_path)
}
end_time <- Sys.time()
end_time - start_time

Time difference of 6.573104 secs

In [8]:
#TMM
process_dataframe <- function(df) {
  counts <- as.matrix(df)
  dge <- DGEList(counts, remove.zeros = TRUE)
  dge <- calcNormFactors(object = dge, method="TMM")
#  dge <- calcNormFactors(object = dge, method="upperquartile")
  normCounts <- cpm(dge, normalized.lib.sizes=TRUE)
  return(as.data.frame(normCounts))
}

In [9]:
start_time <- Sys.time()
norm_data_frames <- list()
for (i in seq_along(data_frames)) {
  file_name <- names(data_frames)[i]
  processed_df <- process_dataframe(data_frames[[i]])
  norm_data_frames[[file_name]] <- processed_df
}
end_time <- Sys.time()
end_time - start_time

Time difference of 6.34944 secs

In [10]:
#change column names in each dataframe to subjectGUID_timepoint

# Load the Metadata
metadata <- read.csv("/home/jupyter/IH-A-Aging-Analysis-Notebooks/Qiuyu-Notebooks/scRNA/hise_meta_data_2024-01-23_fixed.csv")

# Replace empty characters in 'subject.subjectGuid' and 'sample.visitName' with "_"
metadata$subject.subjectGuid <- gsub(" ", "_", metadatasample.visitName <- gsub(" ", "_", metadata$sample.visitName)

# Create a mapping from existing names to new names
name_mapping <- setNames(paste(metadatasample.visitName, sep="_"), metadata$pbmc_sample_id)

# Handle the case where both fields are "_", resulting in "__"
name_mapping[name_mapping == "__"] <- "_"

# Rename Columns in Each DataFrame
for (file_name in names(norm_data_frames)) {
  df <- norm_data_frames[[file_name]]
  # Only rename columns that exist in both the dataframe and the name mapping
  common_names <- intersect(names(df), names(name_mapping))
  names(df)[names(df) %in% common_names] <- name_mapping[common_names]
  norm_data_frames[[file_name]] <- df
}

ERROR: Error in parse(text = x, srcfile = src): <text>:10:1: unexpected symbol
9: # Create a mapping from existing names to new names
10: name_mapping
    ^


ERROR: Error in metadata.head(): could not find function "metadata.head"
